In [1]:
# 디렉토리 관련
import os
import natsort
import os.path
import glob

# 작동 관련
import pandas as pd
import numpy as np
import json
import re

In [2]:
# 비디오 폴더들 전부 나열하기
path = 'C:/Users/admin/Desktop/CAT/RAW/*/**'
video_list = natsort.natsorted( glob.glob(path) )
video_list

['C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201028_cat-arch-000156.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201113_cat-arch-000157.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201114_cat-arch-000925.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201115_cat-arch-001607.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201118_cat-arch-000248.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201118_cat-arch-000458.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201120_cat-arch-001449.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201123_cat-arch-000924.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201125_cat-arch-000928.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201202_cat-arch-003285.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201202_cat-arch-003532.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201202_cat-arch-003817.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201202_cat-arch-004079.mp4',
 'C:/Users/admin/Desktop/CAT/RAW\\ARCH\\20201202_cat-arch-009771.mp4',
 'C:/U

In [3]:
len(video_list)

195

In [4]:
# json 파일들 전부 나열하기
path = 'C:/Users/admin/Desktop/CAT/LABEL/*/**'
json_list = natsort.natsorted( glob.glob(path) )
json_list

['C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201028_cat-arch-000156.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201113_cat-arch-000157.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201114_cat-arch-000925.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201115_cat-arch-001607.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201118_cat-arch-000248.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201118_cat-arch-000458.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201120_cat-arch-001449.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201123_cat-arch-000924.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201125_cat-arch-000928.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201202_cat-arch-003285.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201202_cat-arch-003532.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201202_cat-arch-003817.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL\\ARCH\\20201202_cat-arch-0040

In [5]:
len(json_list)

195

In [6]:
videolist = []
videoname = []
for item in video_list:
    newitem = item.replace('\\', '/')
    videolist.append(newitem)
    name = newitem.split('/')[-1]
    videoname.append(name)

videolist[:2]
# videoname

['C:/Users/admin/Desktop/CAT/RAW/ARCH/20201028_cat-arch-000156.mp4',
 'C:/Users/admin/Desktop/CAT/RAW/ARCH/20201113_cat-arch-000157.mp4']

In [7]:
jsonlist = []
jsonname = []
for item in json_list:
    newitem = item.replace('\\', '/')
    jsonlist.append(newitem)
    name = newitem.split('/')[-1]
    jsonname.append(name)

jsonlist[:2]
# jsonname

['C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201028_cat-arch-000156.mp4.json',
 'C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201113_cat-arch-000157.mp4.json']

In [8]:
# 프레임 넘버를 뽑아내기 위한 정규화 함수 미리 설정
def getFrameNumEx(imgcol):
    p = '_(\d{1,5})_'
    if re.search(p, imgcol):
        result = re.findall(p, imgcol)
        return int( result[0] )

In [18]:
for raw, label in zip(videolist, jsonlist):
    df = pd.DataFrame()
    print(raw, label)
    # image 컬럼 : 이미지 목록 획득
    imgs = natsort.natsorted(glob.glob(f'{raw}/*.jpg'))
    foldername=[]
    for item in imgs:
        newitem = item.split('/')[-1].replace('\\', '/')
        foldername.append(newitem)
    df['image'] = foldername
    df['frame'] = list(map( getFrameNumEx, foldername ))
    # annotation 정보 추출    
    with open( label , encoding='utf-8') as f:
        metaInfo = json.load(f)
    # 실제 이미지가 존재하는 프레임만 추출하여 정보 획득
    check_idx = 0
    keyFrames = list()
    x_cords = list()
    y_cords = list()
    for anno in metaInfo['annotations']:
        if df['frame'][ check_idx ] == anno['frame_number']:
            keyFrames.append( anno['keypoints'] )
            # 세부 좌표 추출
            xs = list()
            ys = list()
            for n in range(1, 15+1):
                tmp = anno['keypoints'][ str(n) ]
                if tmp:
                    xs.append( tmp['x'] )
                    ys.append( tmp['y'] )
                else:
                    xs.append( 0 )
                    ys.append( 0 )   
            x_cords.append( xs )         
            y_cords.append( ys )
            check_idx += 1
            if df.shape[0] == check_idx:
                break
        else:
            continue
        #print( anno['frame_number'], anno['timestamp'] ) 
    #df['keyFrames'] = keyFrames
    df_x_cords = pd.DataFrame(x_cords, columns=[ f'{m}_x' for m in range(15) ])
    df_y_cords = pd.DataFrame(y_cords, columns=[ f'{m}_y' for m in range(15) ])
    # 샘플 1개만 진행, 아래를 제거하면 전체 데이터에 대해서 수행
    f'final_df{' = pd.concat( [df, df_x_cords, df_y_cords], axis=1 )
final_df

C:/Users/admin/Desktop/CAT/RAW/ARCH/20201028_cat-arch-000156.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201028_cat-arch-000156.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201113_cat-arch-000157.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201113_cat-arch-000157.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201114_cat-arch-000925.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201114_cat-arch-000925.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201115_cat-arch-001607.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201115_cat-arch-001607.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201118_cat-arch-000248.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201118_cat-arch-000248.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201118_cat-arch-000458.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201118_cat-arch-000458.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201120_cat-arch-001449.mp4 C:/Users/admin/Desktop/CAT/LABEL/ARCH/20201120_cat-arch-001449.mp4.json
C:/Users/admin/Desktop/CAT/RAW/ARCH/20201

In [19]:
final_df = pd.concat( [df, df_x_cords, df_y_cords], axis=1 )
final_df

,image,frame,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,...,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y,13_y,14_y
0,20201117_cat-armstretch-001283.mp4/frame_0_tim...,0,310,0,385,314,0,216,449,51,...,348,351,507,934,760,686,879,839,0,336
1,20201117_cat-armstretch-001283.mp4/frame_11_ti...,11,382,423,423,375,0,250,473,94,...,323,358,263,934,760,686,879,839,0,357
2,20201117_cat-armstretch-001283.mp4/frame_17_ti...,17,424,454,472,414,0,250,493,101,...,323,401,308,934,760,682,879,839,0,362
3,20201117_cat-armstretch-001283.mp4/frame_101_t...,101,348,362,383,343,0,284,457,292,...,362,366,750,755,0,558,703,679,0,289
4,20201117_cat-armstretch-001283.mp4/frame_107_t...,107,0,0,331,0,0,201,400,174,...,314,324,461,783,650,565,737,706,0,296
5,20201117_cat-armstretch-001283.mp4/frame_113_t...,113,320,0,358,311,0,160,372,28,...,156,299,66,734,650,565,718,706,0,273
6,20201117_cat-armstretch-001283.mp4/frame_119_t...,119,355,0,380,338,0,200,410,0,...,231,374,0,951,671,565,723,706,0,237
7,20201117_cat-armstretch-001283.mp4/frame_125_t...,125,291,392,371,285,0,247,447,207,...,508,549,1000,951,0,0,0,706,0,239
8,20201117_cat-armstretch-001283.mp4/frame_131_t...,131,332,411,377,323,0,255,479,220,...,479,507,947,931,0,0,0,0,0,241
9,20201117_cat-armstretch-001283.mp4/frame_137_t...,137,378,419,419,374,0,281,504,264,...,436,457,903,909,0,609,0,684,0,244


In [20]:
final_df.to_csv('./globfinal.csv')